In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-08-31 00:00:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

In [3]:
# create the vine reviews table
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [7]:
# create dataframe filtering by total votes
vine_votes_df = vine_df.filter(vine_df.total_votes >= 20)
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|     

In [9]:
# filter above dataframe by helpful_votes/total_votes >= 0.5
vine_vote_filter = vine_votes_df.filter(vine_votes_df.helpful_votes/vine_votes_df.total_votes >= 0.5)
vine_vote_filter.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|     

In [10]:
# create another dataframe filtering by vine reviews == "Y"
vine_reviews_df = vine_vote_filter.filter(vine_vote_filter.vine == 'Y')
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|
|R2UUV4UGGYMQG8|          5|           34|         39|   Y|                N|
| R9K0LZV2BK9YY|          4|           37|         39|   Y|                N|
|R2OVFLNEUEGTJM|          3|           18|         25|   Y|                N|
| RBE09ELJ77LQ0|          5|           44|         45|   Y|                N|
|R3867T8AIJJHM6|          5|           26|         27|   Y|                N|
|R1FNVUXPU63WOZ|          4|           43|         48|   Y|                N|
|R25XGG2G12SE1Z|          4|           20|         23|   Y|                N|
| R3JKU4HRDFZDH|          4|           27|         30|   Y|                N|
|R2PQYOCJXRB1BF|          5|           26|         28|   Y|     

In [11]:
# create another dataframe where vien reviews == "N"
vine_reviews_no = vine_vote_filter.filter(vine_vote_filter.vine == 'N')
vine_reviews_no.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|     

## **VINE REVIEWS**

In [14]:
# determine total number of reviews, number of 5-star reviews, and % of 5-star reviews for Vine & non-Vine
# VINE
vine_reviews_df.summary().show()

+-------+--------------+------------------+-----------------+-----------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+-----------------+----+-----------------+
|  count|            47|                47|               47|               47|  47|               47|
|   mean|          null|3.9148936170212765|42.61702127659574|47.91489361702128|null|             null|
| stddev|          null|1.0597330926073165|51.83441579201542|55.06629592837206|null|             null|
|    min|R131OFSBT1U9QT|                 1|               13|               20|   Y|                N|
|    25%|          null|                 3|               20|               23|null|             null|
|    50%|          null|                 4|               26|               30|null|             null|
|    75%|          null|                 5|               44|            

In [15]:
vine_5star = vine_reviews_df.filter(vine_reviews_df.star_rating == 5)
vine_5star.summary().show()

# THERE ARE 15 5-STAR RATINGS FOR PAID VINE REVIEWS OF WATCHES

+-------+--------------+-----------+------------------+------------------+----+-----------------+
|summary|     review_id|star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+-----------+------------------+------------------+----+-----------------+
|  count|            15|         15|                15|                15|  15|               15|
|   mean|          null|        5.0| 35.06666666666667|              38.0|null|             null|
| stddev|          null|        0.0|14.978397142297334|15.496543393378317|null|             null|
|    min|R1B7M0OP3UNP6O|          5|                15|                20|   Y|                N|
|    25%|          null|          5|                23|                24|null|             null|
|    50%|          null|          5|                34|                36|null|             null|
|    75%|          null|          5|                47|                51|null|             null|
|    max| RMEMUGWUX6

In [16]:
15/47 *100

31.914893617021278

## **NON-VINE REVIEWS**

In [17]:
vine_reviews_no.summary().show()

+-------+--------------+------------------+-----------------+-----------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+-----------------+----+-----------------+
|  count|          8362|              8362|             8362|             8362|8362|             8362|
|   mean|          null|3.7969385314518056|42.39619708203779|47.14458263573308|null|             null|
| stddev|          null| 1.526279841658745|73.98106669794147|80.10761823132387|null|             null|
|    min|R100HR2NF8CB9Y|                 1|               10|               20|   N|                N|
|    25%|          null|                 3|               21|               24|null|             null|
|    50%|          null|                 5|               28|               31|null|             null|
|    75%|          null|                 5|               43|            

In [18]:
no_vine_5star = vine_reviews_no.filter(vine_reviews_no.star_rating == 5)
no_vine_5star.summary().show()

+-------+--------------+-----------+------------------+-----------------+----+-----------------+
|summary|     review_id|star_rating|     helpful_votes|      total_votes|vine|verified_purchase|
+-------+--------------+-----------+------------------+-----------------+----+-----------------+
|  count|          4332|       4332|              4332|             4332|4332|             4332|
|   mean|          null|        5.0|45.897737765466296|49.92959372114497|null|             null|
| stddev|          null|        0.0| 91.08640793202062|98.24987525698528|null|             null|
|    min|R102DM895LUN4O|          5|                10|               20|   N|                N|
|    25%|          null|          5|                22|               24|null|             null|
|    50%|          null|          5|                28|               31|null|             null|
|    75%|          null|          5|                45|               48|null|             null|
|    max| RZZMYTB94IFK3|      

In [19]:
4332/8362 *100

51.80578808897393